In [1]:
import optuna
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge  # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb

In [2]:
tokenizer = AutoTokenizer.from_pretrained("EbanLee/kobart-summary-v3")

In [3]:
config_data = {
    "general": {
        "data_path": "../data/", 
        "model_name": "EbanLee/kobart-summary-v3",
        "output_dir": "./"  
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 100,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 50,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": False,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb"
    },
    "wandb": {
        "entity": "legendki",
        "project": "NLP-Summarization",
        "name": "KoBART-summary-v3-optuna",
    },
    "inference": {
        "ckt_path": "model ckt path",
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size": 32,
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

In [4]:
def compute_metrics(config, tokenizer, pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token, " ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token, " ") for sentence in replaced_labels]

    results = rouge.get_scores(replaced_predictions, replaced_labels, avg=True)

    result = {
        'rouge1': results['rouge-1']['f'],
        'rouge2': results['rouge-2']['f'],
        'rougeL': results['rouge-l']['f'],
    }
    return result


In [5]:
def load_trainer_for_train(config, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    training_args = Seq2SeqTrainingArguments(
        output_dir=config['general']['output_dir'],  
        overwrite_output_dir=config['training']['overwrite_output_dir'],
        num_train_epochs=config['training']['num_train_epochs'],
        learning_rate=config['training']['learning_rate'],
        per_device_train_batch_size=config['training']['per_device_train_batch_size'], 
        per_device_eval_batch_size=config['training']['per_device_eval_batch_size'], 
        warmup_ratio=config['training']['warmup_ratio'], 
        weight_decay=config['training']['weight_decay'], 
        lr_scheduler_type=config['training']['lr_scheduler_type'],
        optim=config['training']['optim'],
        gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
        evaluation_strategy=config['training']['evaluation_strategy'], 
        save_strategy=config['training']['save_strategy'],
        save_total_limit=config['training']['save_total_limit'], 
        fp16=config['training']['fp16'],
        load_best_model_at_end=config['training']['load_best_model_at_end'], 
        seed=config['training']['seed'],
        logging_dir=config['training']['logging_dir'], 
        logging_strategy=config['training']['logging_strategy'],
        predict_with_generate=config['training']['predict_with_generate'],
        generation_max_length=config['training']['generation_max_length'],
        do_train=config['training']['do_train'],
        do_eval=config['training']['do_eval'],
        report_to=config['training']['report_to']
    )

    wandb.init(
        entity=config['wandb']['entity'],
        project=config['wandb']['project'],
        name=config['wandb']['name'],
    )

    os.environ["WANDB_LOG_MODEL"] = "true"
    os.environ["WANDB_WATCH"] = "false"

    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    trainer = Seq2SeqTrainer(
        model=generate_model,  
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics=lambda pred: compute_metrics(config, tokenizer, pred),
        callbacks=[MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [6]:
def load_tokenizer_and_model_for_train(config, device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)
    model_name = config['general']['model_name']
    bart_config = BartConfig().from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'], config=bart_config)

    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model.resize_token_embeddings(len(tokenizer))  
    generate_model.to(device)
    print(generate_model.config)

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)
    return generate_model, tokenizer

In [7]:
def load_data(config):
    data_path = config['general']['data_path']
    train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
    val_df = pd.read_csv(os.path.join(data_path, 'dev.csv'))
    return train_df, val_df

In [8]:
class Preprocess:
    def __init__(self, bos_token: str, eos_token: str):
        self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    def make_set_as_df(file_path, is_train=True):
        df = pd.read_csv(file_path)
        if is_train:
            return df[['fname', 'dialogue', 'summary']]
        else:
            return df[['fname', 'dialogue']]

    def make_input(self, dataset, is_test=False):
        if is_test:
            encoder_input = dataset['dialogue']
            decoder_input = [self.bos_token] * len(dataset['dialogue'])
            return encoder_input.tolist(), list(decoder_input)
        else:
            encoder_input = dataset['dialogue']
            decoder_input = dataset['summary'].apply(lambda x: self.bos_token + str(x))
            decoder_output = dataset['summary'].apply(lambda x: str(x) + self.eos_token)
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()


In [9]:
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    train_file_path = os.path.join(data_path, 'train.csv')
    val_file_path = os.path.join(data_path, 'dev.csv')

    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    print('-'*150)
    print(f'train_data:\n {train_data["dialogue"][0]}')
    print(f'train_label:\n {train_data["summary"][0]}')

    print('-'*150)
    print(f'val_data:\n {val_data["dialogue"][0]}')
    print(f'val_label:\n {val_data["summary"][0]}')

    encoder_input_train, decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val, decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)
    print('-'*10, 'Load data complete', '-'*10, )

    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                                         add_special_tokens=True, truncation=True, max_length=config['tokenizer'][
            'encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                                         add_special_tokens=True, truncation=True, max_length=config['tokenizer'][
            'decoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_outputs = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                                          add_special_tokens=True, truncation=True, max_length=config['tokenizer'][
            'decoder_max_len'], return_token_type_ids=False)

    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_outputs,
                                           len(encoder_input_train))

    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                                             add_special_tokens=True, truncation=True, max_length=config['tokenizer'][
            'encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                                             add_special_tokens=True, truncation=True, max_length=config['tokenizer'][
            'decoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_outputs = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                                              add_special_tokens=True, truncation=True, max_length=config['tokenizer'][
            'decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs,
                                       val_tokenized_decoder_outputs, len(encoder_input_val))

    print('-'*10, 'Make dataset complete', '-'*10, )
    return train_inputs_dataset, val_inputs_dataset


In [10]:
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()}  
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) 
        item['labels'] = self.labels['input_ids'][idx] 
        return item

    def __len__(self):
        return self.len

In [11]:
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}  
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} 
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) 
        item['labels'] = self.labels['input_ids'][idx] 
        return item

    def __len__(self):
        return self.len

In [12]:
def objective(trial):
    config_data['training']['learning_rate'] = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)
    config_data['training']['per_device_train_batch_size'] = trial.suggest_categorical('per_device_train_batch_size', [4, 8, 16, 32])
    config_data['training']['num_train_epochs'] = trial.suggest_int('num_train_epochs', 10, 30)
    config_data['training']['warmup_ratio'] = trial.suggest_float('warmup_ratio', 0.0, 0.3)
    config_data['training']['optim'] = trial.suggest_categorical('optim', ['adamw_torch', 'adamw_hf', 'adafactor'])
    config_data['training']['gradient_accumulation_steps'] = trial.suggest_int('gradient_accumulation_steps', 1, 4)
    config_data['training']['lr_scheduler_type'] = trial.suggest_categorical('lr_scheduler_type', ['linear', 'cosine'])
    config_data['training']['fp16'] = trial.suggest_categorical('fp16', [False])

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f"Running on: {device}")

    generate_model, tokenizer = load_tokenizer_and_model_for_train(config_data, device)
    print("Model and Tokenizer Loaded.")

    preprocessor = Preprocess(config_data['tokenizer']['bos_token'], config_data['tokenizer']['eos_token'])
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config_data, preprocessor, config_data['general']['data_path'], tokenizer)

    trainer = load_trainer_for_train(config_data, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset)

    trainer.train()

    eval_metrics = trainer.evaluate(eval_dataset=val_inputs_dataset)
    rougeL = eval_metrics.get('rougeL', 0.0)

    return rougeL

In [13]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print(f"Best trial: {study.best_trial.value}")
print(f"Best parameters: {study.best_trial.params}")

[I 2024-08-30 08:23:20,988] A new study created in memory with name: no-name-1fedfa8c-3f14-4c5b-9198-f8ddf62bed20
/tmp/ipykernel_89799/3998167599.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  config_data['training']['learning_rate'] = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Running on: cuda:0
---------- Load tokenizer & model ----------
---------- Model Name : EbanLee/kobart-summary-v3 ----------
BartConfig {
  "_name_or_path": "EbanLee/kobart-summary-v3",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "EbanLee(rudwo6769@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": fals

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


---------- Make dataset complete ----------
---------- Make training arguments ----------


wandb: Currently logged in as: rmadyd0314 (legendki). Use `wandb login --relogin` to force relogin


...

---------- Make training arguments complete ----------
---------- Make trainer ----------
---------- Make trainer complete ----------


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,0.810200,0.593800,0.353104,0.119105,0.337299
2,0.566400,0.549556,0.370176,0.132408,0.355062
3,0.504800,0.530380,0.371765,0.136098,0.355966
4,0.443500,0.525955,0.382022,0.146392,0.364857
5,0.390600,0.528711,0.384217,0.148396,0.366739
6,0.346900,0.543156,0.381277,0.147669,0.366788
7,0.309100,0.557211,0.377228,0.143517,0.360896


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


[I 2024-08-30 13:04:32,241] Trial 9 finished with value: 0.0 and parameters: {'learning_rate': 1.996091241653689e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 19, 'warmup_ratio': 0.13691082569395924, 'optim': 'adamw_torch', 'gradient_accumulation_steps': 2, 'lr_scheduler_type': 'linear', 'fp16': False}. Best is trial 0 with value: 0.0.


Best trial: 0.0
Best parameters: {'learning_rate': 2.1169083140275866e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 26, 'warmup_ratio': 0.09577831393575928, 'optim': 'adamw_hf', 'gradient_accumulation_steps': 3, 'lr_scheduler_type': 'cosine', 'fp16': False}


In [12]:
def objective(trial):
    # Optuna가 제공하는 hyperparameters를 사용하여 config 업데이트
    config_data['training']['learning_rate'] = 2.1169083140275866e-05
    config_data['training']['per_device_train_batch_size'] = 8
    config_data['training']['num_train_epochs'] = 26
    config_data['training']['warmup_ratio'] = 0.09577831393575928
    config_data['training']['optim'] = 'adamw_hf'
    config_data['training']['gradient_accumulation_steps'] = 3
    config_data['training']['lr_scheduler_type'] = 'cosine'
    config_data['training']['fp16'] = False

    # Device 설정
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f"Running on: {device}")

    # 모델 및 tokenizer 불러오기
    generate_model, tokenizer = load_tokenizer_and_model_for_train(config_data, device)
    print("Model and Tokenizer Loaded.")

    # 데이터 로드 및 전처리
    preprocessor = Preprocess(config_data['tokenizer']['bos_token'], config_data['tokenizer']['eos_token'])
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config_data, preprocessor, config_data['general']['data_path'], tokenizer)

    # Trainer 설정
    trainer = load_trainer_for_train(config_data, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset)

    # Training 시작
    trainer.train()

    # Validation 성능을 기준으로 최적의 하이퍼파라미터 선택
    eval_metrics = trainer.evaluate(eval_dataset=val_inputs_dataset)
    rougeL = eval_metrics.get('rougeL', 0.0)

    return rougeL

In [13]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Running on: {device}")

# 모델 및 tokenizer 불러오기
generate_model, tokenizer = load_tokenizer_and_model_for_train(config_data, device)
print("Model and Tokenizer Loaded.")

# 데이터 로드 및 전처리
preprocessor = Preprocess(config_data['tokenizer']['bos_token'], config_data['tokenizer']['eos_token'])
train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config_data, preprocessor, config_data['general']['data_path'], tokenizer)

# Trainer 설정
trainer = load_trainer_for_train(config_data, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset)

# Training 시작
trainer.train()

# Save the best model checkpoint path to the config
best_model_path = trainer.state.best_model_checkpoint
config_data['inference']['ckt_path'] = best_model_path

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Running on: cuda:0
---------- Load tokenizer & model ----------
---------- Model Name : EbanLee/kobart-summary-v3 ----------
BartConfig {
  "_name_or_path": "EbanLee/kobart-summary-v3",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "EbanLee(rudwo6769@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": fals

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


---------- Make dataset complete ----------
---------- Make training arguments ----------


wandb: Currently logged in as: rmadyd0314 (legendki). Use `wandb login --relogin` to force relogin


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


---------- Make training arguments complete ----------
---------- Make trainer ----------
---------- Make trainer complete ----------


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,1.022000,0.634309,0.326655,0.101321,0.312527
2,0.608200,0.581463,0.351159,0.117514,0.336022
3,0.558600,0.556011,0.364429,0.128457,0.346924
4,0.522500,0.544667,0.367032,0.132963,0.348829
5,0.496800,0.538125,0.371644,0.135485,0.352777
6,0.475900,0.534033,0.372839,0.135696,0.354633
7,0.456100,0.532377,0.370061,0.132959,0.352563
8,0.439200,0.530600,0.374595,0.138304,0.358497
9,0.423800,0.531613,0.376204,0.143367,0.361171
10,0.410900,0.533131,0.374713,0.140592,0.358827


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [16]:
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, len):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.len

In [17]:
def prepare_test_dataset(config,preprocessor, tokenizer):

    test_file_path = os.path.join(config['general']['data_path'],'test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path,is_train=False)
    test_id = test_data['fname']

    print('-'*150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-'*150)

    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data,is_test=True)
    print('-'*10, 'Load data complete', '-'*10,)

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10,)

    return test_data, test_encoder_inputs_dataset

In [18]:
def load_tokenizer_and_model_for_test(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)

    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('-'*10, f'Model Name : {model_name}', '-'*10,)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model = BartForConditionalGeneration.from_pretrained(ckt_path)
    generate_model.resize_token_embeddings(len(tokenizer))
    generate_model.to(device)
    print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model , tokenizer

In [19]:
def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    generate_model , tokenizer = load_tokenizer_and_model_for_test(config,device)

    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config,preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output.csv"), index=False)

    return output

In [20]:

def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    generate_model , tokenizer = load_tokenizer_and_model_for_test(config,device)

    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config,preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output.csv"), index=False)

    return output

In [21]:
if __name__ == "__main__":
    output = inference(config_data)
    print(output)

---------- device : cuda:0 ----------
2.1.0
---------- Load tokenizer & model ----------
---------- Model Name : EbanLee/kobart-summary-v3 ----------


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.
#Person2#: 그게 다신가요?
#Person1#: 네. 

100%|██████████| 16/16 [00:23<00:00,  1.46s/it]

        fname                                            summary
0      test_0     더슨 씨는 #Person1# 에게 모든 직원에게 내부 메모가 전달되어야 한다고...
1      test_1    #Person1# 은 교통 체증에 걸렸다. #Person2# 는 #Person1...
2      test_2     케이트는 마샤와 히어로가 2개월 동안 별거 중이다가 이혼을 신청했다고 #Per...
3      test_3    #Person1# 은 브라이언의 생일을 축하하기 위해 파티를 즐긴다. 브란드는 ...
4      test_4    #Person2# 는 #Person1# 에게 올림픽 스타디움에 있는 올림픽 공원...
..        ...                                                ...
494  test_495     잭이 찰리에게 새 게임에 대해 묻습니다. 찰리는 잭에게 캐릭터를 만드는 게임을...
495  test_496    #Person2# 는 #Person1# 에게 #Person2# 가 컨트리 음악 ...
496  test_497     앨리스는 #Person1# 에게 세탁기, 건조기, 비누를 어떻게 사용하는지 가...
497  test_498     스티브는 매튜에게 그녀의 계약이 다음 달에 끝나기 때문에 최근에 살 곳을 찾고...
498  test_499     프랭크는 벳시에게 승진하고 친구들 모두를 위한 큰 파티를 열 계획이라고 말한다...

[499 rows x 2 columns]


In [23]:
output_path = os.path.join(config_data['inference']['result_path'], "output.csv")
output_df = pd.read_csv(output_path)
print(output_df.head())

    fname                                            summary
0  test_0     더슨 씨는 #Person1# 에게 모든 직원에게 내부 메모가 전달되어야 한다고...
1  test_1    #Person1# 은 교통 체증에 걸렸다. #Person2# 는 #Person1...
2  test_2     케이트는 마샤와 히어로가 2개월 동안 별거 중이다가 이혼을 신청했다고 #Per...
3  test_3    #Person1# 은 브라이언의 생일을 축하하기 위해 파티를 즐긴다. 브란드는 ...
4  test_4    #Person2# 는 #Person1# 에게 올림픽 스타디움에 있는 올림픽 공원...
